# Part A: Probabilistic Gaussian Generative Classifier

## A1. Dataset and Setup

In [4]:
import sklearn as sk
from sklearn.datasets import load_digits
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import numpy as np

Load dataset

In [5]:
digits = load_digits()
X,y = digits.data, digits.target

In [6]:
X.shape, y.shape

((1797, 64), (1797,))

Split dataset into 70% and 30%

In [7]:
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.30, stratify=y, random_state=42
)

Split 30% into 15% calidation and 15% test

In [8]:
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.50, stratify=y_temp, random_state=42
)

Standarize features

In [9]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled   = scaler.transform(X_val)
X_test_scaled  = scaler.transform(X_test)

## A2. Gaussian Generative Model

In [10]:
class GaussianGenerativeModel():
    def __init__(self,x_train,y_train):
        self.x_train = x_train
        self.y_train = y_train
        self.d = self.x_train.shape[1]
        self.pi = np.zeros(10) # Class priors
        self.mu = np.zeros((10, self.d)) # Class means
        self.sigma = np.zeros((self.d, self.d)) # Covariance matrix
        self.sigma_reg = np.zeros((self.d, self.d)) # Regularized covariance matrix
    def _estimate_class_priors(self):
        n = len(self.y_train)
        for k in range(10):
            pi_k = np.sum(self.y_train == k)/n
            self.pi[k] = pi_k
        return self.pi
    def _estimate_class_means(self):
        for k in range(10):
            x_class = self.x_train[self.y_train == k] # Get all samples that are equal to k
            mu_k = x_class.mean(axis=0) # return a vector of size 64x1
            self.mu[k] = mu_k
    def _estimate_covariance_matrix(self):
        self.sigma = np.zeros((self.d, self.d)) 
        # Iterate over training samples
        for i in range(len(self.y_train)):
            # compute the difference
            diff = self.x_train[i] - self.mu[self.y_train[i]]
            # create a diagonal matrix
            self.sigma += np.outer(diff,diff)
        self.sigma /= len(self.y_train)
    def regularize(self,reg_term=0):
        self.sigma_reg = self.sigma + reg_term * np.eye(self.d)
    def fit(self, reg_term=0):
        self._estimate_class_priors()
        self._estimate_class_means()
        self._estimate_covariance_matrix()
        self.regularize(reg_term)
    def _predict(self,x):
        scores =np.zeros(10)
        for k in range(10):
            diff = x - self.mu[k]
            score = np.log(self.pi[k]) - 0.5 * (diff.T @ np.linalg.inv(self.sigma_reg) @ diff)
            scores[k]= score
        prediction = np.argmax(scores)
        return prediction
    def predict(self,x_test):
        predictions = [self._predict(x) for x in x_test]
        return predictions

            

Create an instance of the model

In [12]:
model = GaussianGenerativeModel(X_train_scaled , y_train)

In [15]:
model.fit( 1e-4 )

In [19]:
predictions = model.predict(X_test_scaled)

## A3. Hyperparameter Tuning and Evaluation

In [18]:
reg_term_values = [1e-4, 1e-3, 1e-2, 1e-1]

In [21]:
for reg in reg_term_values:
    model.fit(reg)
    predictions =  model.predict(X_test_scaled)
    # TODO: calculate accuracy and get best value
    # TODO: implement metrics evaluation

[np.int64(6), np.int64(1), np.int64(8), np.int64(1), np.int64(3), np.int64(7), np.int64(6), np.int64(6), np.int64(8), np.int64(0), np.int64(7), np.int64(1), np.int64(9), np.int64(3), np.int64(5), np.int64(5), np.int64(7), np.int64(8), np.int64(3), np.int64(9), np.int64(6), np.int64(3), np.int64(1), np.int64(6), np.int64(6), np.int64(2), np.int64(2), np.int64(9), np.int64(8), np.int64(6), np.int64(7), np.int64(0), np.int64(9), np.int64(2), np.int64(5), np.int64(7), np.int64(2), np.int64(1), np.int64(4), np.int64(3), np.int64(5), np.int64(1), np.int64(8), np.int64(8), np.int64(3), np.int64(3), np.int64(5), np.int64(7), np.int64(5), np.int64(4), np.int64(6), np.int64(3), np.int64(6), np.int64(7), np.int64(2), np.int64(1), np.int64(4), np.int64(1), np.int64(1), np.int64(4), np.int64(3), np.int64(4), np.int64(6), np.int64(5), np.int64(4), np.int64(6), np.int64(7), np.int64(2), np.int64(2), np.int64(4), np.int64(5), np.int64(0), np.int64(3), np.int64(3), np.int64(0), np.int64(3), np.int64(0)